In [21]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from standardiser import standardise
import csv

df = pd.read_csv('../raw/Selin-All.csv')
df = df[df["Active"] == 1].sort_values("Avg-B-Score", ascending=True).head(100)
mols = [Chem.MolFromSmiles(smi) for smi in df["SMILES"]]

std_mols = []
for mol in mols:
    try:
        mol = standardise.run(mol)
        std_mols.append(mol)
    except:
        continue

def is_interesting(mol):
    mw = Chem.Descriptors.MolWt(mol)
    if mw < 300:
        return False
    if mw > 800:
        return False
    return True

std_mols = [mol for mol in std_mols if is_interesting(mol)]

std_smiles = [Chem.MolToSmiles(mol) for mol in std_mols]
with open('../atb_screening_data_top100.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['inchikey', 'smiles'])
    for smiles in std_smiles:
        inchikey = Chem.MolToInchiKey(Chem.MolFromSmiles(smiles))
        writer.writerow([inchikey, smiles])